In [1]:
import argparse
import sys
import subprocess

In [1]:
import gwsurrogate
import numpy as np

from math import cos
from math import sin
from math import sqrt
from math import factorial
from math import pi
from math import e
from numpy import conj

/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


In [8]:
q = 1.5
chiA = [0,0,0]
chiB = [0,0,0]
times = np.array([-1000, -500])
f_low = 0
sur = gwsurrogate.LoadSurrogate('NRHybSur3dq8')
t, h, dyn = sur(q, chiA, chiB, times=times, f_low=f_low)

Loaded NRHybSur3dq8 model


In [13]:
h

{(2, 2): array([0.02164953-0.12617358j, 0.14943634+0.j        ]),
 (3, 2): array([0.00017178-9.86785364e-04j, 0.00138251+3.93822684e-05j]),
 (4, 4): array([ 0.00213649+0.00082025j, -0.00310821-0.0001292j ]),
 (3, 3): array([ 0.00475882-0.00307026j, -0.00040963-0.00710687j]),
 (3, 0): array([0.+2.04210616e-05j, 0.+5.82452503e-06j]),
 (3,
  1): array([-1.17349190e-04-8.30497436e-05j,  4.42436745e-05+1.69200014e-04j]),
 (2, 1): array([-0.00192393-0.00210577j,  0.00019859+0.00362907j]),
 (2, 0): array([ 2.48516314e-05+0.j, -1.23912955e-07+0.j]),
 (4,
  3): array([ 9.50328340e-05-6.49684142e-05j, -9.83783895e-06-1.72384512e-04j]),
 (4,
  2): array([-2.99112385e-06-1.01882613e-04j,  1.34085965e-04-2.70289933e-05j]),
 (5, 5): array([ 8.08912927e-05+0.00023015j, -1.35541745e-05+0.00035498j])}

In [4]:
# Define constants: speed of light (in m/s), gravitational constant (in m^3/(kg*s^2)), solar mass (in kg) and megaparsec (in m)
c = 2.99e8
G = 6.67e-11
M_sun = 1.99e30
Mpc = 3.09e22

In [ ]:
class Fn2:
    def __init__(v_mag_, v_):
        
        self.v_mag_ = v_mag_
        self.v_ = v_
    
    # Define delta, binomial coeffcient, A, F and G functions
    def delta_(self,a,b):
        if a == b:
            return True
        else:
            return False

    def binomial_(self,n,k):
        if n < 0 or k < 0 or n-k < 0:
            return 0
        else:
            return factorial(n)/(factorial(k)*factorial(n-k))

    def A_(self,l,m):
        if (l-m)*(l+m+1) > 0 and (l+m)*(l-m+1) > 0:
            return sqrt((l-m)*(l+m+1)) - sqrt((l+m)*(l-m+1))
        elif (l-m)*(l+m+1) > 0:
            return sqrt((l-m)*(l+m+1))
        elif (l+m)*(l-m+1) > 0:
            return - sqrt((l+m)*(l-m+1))
        else:
            return 0

    def F_(self,l1,m1,l2,m2):
        if l1 < abs(m1) or l2 < abs(m2):
            return 0
        else:
            return sqrt(factorial(l1+m1)*factorial(l1-m1)*(2*l1+1)/(factorial(l1-2)*factorial(l1+2)))*sqrt(factorial(l2+m2)*factorial(l2-m2)*(2*l2+1)/(factorial(l2-2)*factorial(l2+2)))

    def G_(self,l1,m1,k1,a1,b1,l2,m2,k2,a2,b2):
        return binomial(l1+2,k1)*binomial(l1-2,k1-m1-2)*binomial(2*l1-2*k1+m1+2,a1)*binomial(2*k1-m1-2,b1)*binomial(l2-2,k2)*binomial(l2+2,k2+m2+2)*binomial(2*l2-2*k2-m2-2,a2)*binomial(2*k2+m2+2,b2)


    # Define mode coefficient functions
    def C0_(self,l1,l2,m):
        v_mag = self.v_mag_
        v = self.v_
        
        hlp = 0

        for k1 in range(max(0,m+2),l1+3):
            for a1 in range(2*l1-2*k1+m+3):
                for b1 in range(2*k1-m-1):
                    for k2 in range(max(0,m-2),l2-1):
                        for a2 in range(2*l2-2*k2-m-1):
                            for b2 in range(2*k2+m+3):
                                u = l1+l2-a1-a2-b1-b2
                                if delta(u,0):
                                    hlp = hlp + (-1)**(a1+a2)*m*G(l1,m,k1,a1,b1,l2,m,k2,a2,b2)

        return 2*v_mag*pi*1j*v[2]*F(l1,m,l2,m)*hlp

    def Cp_(self,l1,l2,m):
        v_mag = self.v_mag_
        v = self.v_
        
        hlp = 0

        for k1 in range(max(0,m+3),l1+3):
            for a1 in range(2*l1-2*k1+m+4):
                for b1 in range(2*k1-m-2):
                    for k2 in range(max(0,m-2),l2-1):
                        for a2 in range(2*l2-2*k2-m-1):
                            for b2 in range(2*k2+m+3):
                                u = l1+l2-a1-a2-b1-b2

                                if delta(abs(u%2),1):
                                    hlp = hlp + (-1)**(a1+a2)*G(l1,m+1,k1,a1,b1,l2,m,k2,a2,b2)*2*A(l1,m+1)/(u*(u**2-4))
                                elif delta(u,2):
                                    hlp = hlp + (-1)**(a1+a2)*G(l1,m+1,k1,a1,b1,l2,m,k2,a2,b2)*(m-1)*pi/4
                                elif delta(u,-2):
                                    hlp = hlp - (-1)**(a1+a2)*G(l1,m+1,k1,a1,b1,l2,m,k2,a2,b2)*(m-1)*pi/4

        return (1j*v[0]-v[1])*v_mag*F(l1,m+1,l2,m)*hlp

    def Cm_(self,l1,l2,m):
        v_mag = self.v_mag_
        v = self.v_
        
        hlp = 0

        for k1 in range(max(0,m+1),l1+3):
            for a1 in range(2*l1-2*k1+m+2):
                for b1 in range(2*k1-m):
                    for k2 in range(max(0,m-2),l2-1):
                        for a2 in range(2*l2-2*k2-m-1):
                            for b2 in range(2*k2+m+3):
                                u = l1+l2-a1-a2-b1-b2

                                if delta(abs(u%2),1):
                                    hlp = hlp + (-1)**(a1+a2)*G(l1,m-1,k1,a1,b1,l2,m,k2,a2,b2)*2*A(l1,m-1)/(u*(u**2-4))
                                elif delta(u,2):
                                    hlp = hlp - (-1)**(a1+a2)*G(l1,m-1,k1,a1,b1,l2,m,k2,a2,b2)*(m-1)*pi/4
                                elif delta(u,-2):
                                    hlp = hlp + (-1)**(a1+a2)*G(l1,m-1,k1,a1,b1,l2,m,k2,a2,b2)*(m-1)*pi/4

        return (1j*v[0]+v[1])*v_mag*F(l1,m-1,l2,m)*hlp
    
    
    def Coefficient(self):
        v_mag = self.v_mag_
        v = self.v_
        
        #calling functions
        delta = self.delta_
        binomial = self.binomial_
        A = self.A_
        F = self.F_
        G = self.G_
        C0 = self.C0_
        Cp = self.Cp_
        Cm = self.Cm_
        
        # Set maximal 'l' available
        l = 5
        
        # Define arrays to save the coefficients
        coeff = np.zeros((3,l+1,l+1,2*(l+1)), dtype=complex)

        # Compute and save the coefficients
        for l1 in range(2,l+1):
            for l2 in range(2,l+1):
                for m in range(-l2,l2+1):
                    coeff[0][l1][l2][m] = C0(l1,l2,m)
                    coeff[1][l1][l2][m] = Cp(l1,l2,m)
                    coeff[2][l1][l2][m] = Cm(l1,l2,m)

                    
        return(coeff[0] ,coeff[1] ,coeff[2])

In [14]:
class Fn:
    def __init_(self, M_, q_, dis_, the_, phi_, v_mag_, v_the_, v_phi, times_):
        
        self.M_ = M_*M_sun
        self.q_ = q_
        self.dis_ = dis_*Mpc
        self.the_ = pi*float(the_)/180
        self.phi_ = pi*float(phi_)/180
        self.v_mag_ = float(v_mag_)/c
        self.v_the_ = pi*float(v_the_)/180
        self.v_phi_ = pi*float(v_phi_)/180
        self.times_ = times_
    
    # Define binomial function and spherical harmonics
    def binomial_(self, n, k):
        if n < 0 or k < 0 or n-k < 0:
            return 0
        else:
            return factorial(n)/(factorial(k)*factorial(n-k))
            
    def Y_(self, l, m, THE, PHI, S):
        hlp = 0
        for k in range(max(0,m-s),l-s+1):
            if binomial(l-s,k) != 0 and binomial(l+s,k+s-m) != 0:
                hlp = hlp + (-1)**(l-k-s+m)*binomial(l-s,k)*binomial(l+s,k+s-m)*\
                cos(THE/2)**(2*k+s-m)*sin(THE/2)**(2*l-2*k-s+m)*e**(1j*m*PHI)
                
                return sqrt(factorial(l+m)*factorial(l-m)*(2*l+1)/\
                            (4*pi*factorial(l+s)*factorial(l-s)))*hlp
            
    def WaVe(self):
        M = self.M_
        q = self.q_
        dis = self.dis_
        the = self.the_
        phi = self.the_
        v_mag = self.v_mag_
        v_the = self.v_the_
        v_phi = self.v_phi_
        times = self.times_
        s = -2
        Y = self.Y_
        binomial = self.binomial_
        
        
        chiA = [0,0,0]
        chiB = [0,0,0]
        
        v = np.array([sin(v_the)*cos(v_phi),sin(v_the)*sin(v_phi),cos(v_the)])
        
        times = np.array(times)

        # Compute the Doppler shift and the amplitude of the wave
        Dopp = (1+v_mag*(sin(the)*cos(phi)*v[0]+sin(the)*sin(phi)*v[1]+ \
                         cos(the)*v[2]))/sqrt(1-v_mag**2)
        amp = G*M/(dis*c**2)
        
        # Transform the times for evaluation in natural units and set f_low to zero so that it can in principle compute all times
        for i in range(len(times)):
            times[i] = c**3*times[i]/(G*M*Dopp)

        f_low = 0
        
        # Load the surrogate model
        sur = gwsurrogate.LoadSurrogate('NRHybSur3dq8')
        
        # 'Get' the wave form
        t, h, dyn = sur(q, chiA, chiB, times=times, f_low=f_low)
        
        # Compute and write out the spherical harmonics
        y = np.zeros((6,12), dtype=complex)
        
        for l2 in range(2,6):
            for m in range(-l2,l2+1):
                y[l2][m] = Y(l2,m,the,phi,s)
        
        #calling the class for coeff
        coeff_ = Fn2(v_mag, v)
        c0, cp, cm = coeff_.Coefficient()
        
        # Compute the beamed modes
        # Create array for beamed modes
        hb = {}
        for mi in h:
            hb[mi] = [0]*len(t)
            hb[(mi[0],-mi[1])] = [0]*len(t)
        
        
        # Compute the beamed modes over time
        for mi in hb:
            for i in range(len(t)):
                # Compute the contributions of each mode
                hlp0 = 0
                for l0 in range(max(2,abs(mi[1])),6):
                    try:
                        h[(l0,abs(mi[1]))]
                    except KeyError:
                        pass
                    else:
                        if mi[1] < 0:
                            hlp0 = hlp0 + (1/4**l0)*(-1)**l0*conj(h[(l0,abs(mi[1]))][i])*c0[l0][mi[0]][mi[1]]
                        else:
                            hlp0 = hlp0 + (1/4**l0)*h[(l0,mi[1])][i]*c0[l0][mi[0]][mi[1]]

                hlpp = 0
                for lp in range(max(2,abs(mi[1]+1)),6):
                    try:
                        h[(lp,abs(mi[1]+1))]
                    except KeyError:
                        pass
                    else:
                        if mi[1]+1 < 0:
                            hlpp = hlpp + (1/4**lp)*(-1)**lp*conj(h[(lp,abs(mi[1]+1))][i])*cp[lp][mi[0]][mi[1]]
                        else:
                            hlpp = hlpp + (1/4**lp)*h[(lp,mi[1]+1)][i]*cp[lp][mi[0]][mi[1]]

                hlpm = 0
                for lm in range(max(2,abs(mi[1]-1)),6):
                    try:
                        h[(lm,abs(mi[1]-1))]
                    except KeyError:
                        pass
                    else:
                        if mi[1]-1 < 0:
                            hlpm = hlpm + (1/4**lm)*(-1)**lm*conj(h[(lm,abs(mi[1]-1))][i])*cp[lp][mi[0]][mi[1]]
                        else:
                            hlpm = hlpm + (1/4**lm)*h[(lm,mi[1]-1)][i]*cm[lm][mi[0]][mi[1]]

                # Sum up the contributions to get the exited mode
                if mi[1] < 0:
                    hb[mi][i] = (-1)**mi[0]*conj(h[(mi[0],abs(mi[1]))][i]) + (-1)**abs(mi[1])*(1j/4**(mi[0]+1))*(hlp0+hlpp+hlpm)
                else:
                    hb[mi][i] = h[mi][i] + (-1)**mi[1]*(1j/4**(mi[0]+1))*(hlp0+hlpp+hlpm)


        # Define arrays for the waveforms
        hp = [0]*len(t)
        hc = [0]*len(t)
        
        # Compute polarizations over time
        for i in range(len(t)):

            hlp = 0
            for mi in hb:
                hlp = hlp + hb[mi][i]*y[mi[0]][mi[1]]

            hp[i], hc[i] = hlp.real, hlp.imag

            # Convert time and amplitude in 'SI' units
            hp[i], hc[i] = amp*hp[i], amp*hc[i]


        # Write out the waveforms
        return(hp,hc)
               

In [ ]:
import bilby
import numpy as np

In [ ]:
# Define labels, sampling frequency and times
outdir = 'outdir'
label = 'moving_bbh'
sampling_frequency = 1024
duration = 2.0
geocent_time = 0.0
start_time = geocent_time - duration + 0.02

In [15]:
# Define the time-domain model
def moving_bbh(times, mass, ratio, distance, theta, phi, speed, v_the, v_phi):

    arg_ = {'M_':mass, 'q_':ratio, 'dis_':distance, 'the_':theta, 'phi_':phi, 'v_mag_':speed, 'v_the_':v_the, 'v_phi':v_phi, 'times_':times}
    fplus = Fn(**arg_)
    h_plus = fplus.WaVe()[0]
    h_cross = fplus.WaVe()[1]

    return {'plus': h_plus, 'cross': h_cross}

In [ ]:
# Define parameters to inject
injection_parameters = dict(mass=50, ratio=6, distance=300, theta=45, phi=45, speed=3000, v_the=0, v_phi=0, ra=0, dec=0, psi=0, geocent_time=geocent_time)
# Generate model with waveform_generator
waveform = bilby.gw.waveform_generator.WaveformGenerator(sampling_frequency=sampling_frequency, duration=duration, time_domain_source_model=moving_bbh, start_time=start_time)
# Inject signal into ifos
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1', 'V1'])
ifos.set_strain_data_from_power_spectral_densities(sampling_frequency=sampling_frequency, duration=duration, start_time=start_time)
ifos.inject_signal(waveform_generator=waveform, parameters=injection_parameters)

In [ ]:
# Create priors
prior = injection_parameters.copy()
prior['mass'] = bilby.core.prior.Uniform(45, 55, r'$M$', unit='$M_sun$')

# Define likelihood
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(interferometers=ifos, waveform_generator=waveform, priors=prior)

In [ ]:
# Launch sampler
result = bilby.core.sampler.run_sampler(likelihood, prior, sampler='dynesty', npoints=500, injection_parameters=injection_parameters, outdir=outdir, label=label, dlogz=3, npool=6)